In [1]:
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
?urllib.request

In [2]:
def get_page(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    mainpage = response.read().decode('utf-8')
    return mainpage

# Get all list of URL

In [3]:
# Navigate to Main Page
mainpage = get_page('https://www.hipflat.co.th/en/bangkok/condo')
mainpage_parser = BeautifulSoup(mainpage,'html.parser')

In [ ]:
# Find hyperlink from the main page base on list of districts shown on page
district_url = []
for link in mainpage_parser.find_all("a",{"class":"directories__lists-element-name"}):
    district_url.append('https://www.hipflat.co.th' + str(link.get('href')))

district_url

In [ ]:
# Navigate to each district page and fetch all condos hyperlink
condo_url = []
for district in district_url:
    districtpage_parser = BeautifulSoup(get_page(district),'html.parser')
    for link in districtpage_parser.find_all("a",{"class":"directories__lists-element-name"}):
        condo_url.append('https://www.hipflat.co.th' + str(link.get('href')))

condo_url

# Backup URL

In [ ]:
import pickle
# Backup Datasets for Modeling
with open('HipFlat_URL.pickle', 'wb') as f:
    pickle.dump([district_url,condo_url], f)

In [5]:
import pickle
# Load Datasets for Modeling
with open('HipFlat_URL.pickle', 'rb') as f:
    w = pickle.load(f)
    
district_url = w[0]
condo_url = w[1]

In [ ]:
condo_url

# Fetch Information from Each Page

In [ ]:
import pandas
from pandas import DataFrame
import re
#Create Empty DataFrame for Keeping Fetched Data
condo_master_profile = pandas.DataFrame(columns=['name','district','province','address', \
                                                 'lat','lng','floor','tower','yearbuilt', \
                                                 'current_asking_price_per_sqm','asking_price_chg_yoy','gross_rental_yield','units', \
                                                 'nearest_bts','bts_distance','bts_lat','bts_lng', \
                                                 'nearest_mrt','mrt_distance','mrt_lat','mrt_lng', \
                                                 'nearest_xpress_way','xpress_way_distance','xpress_lat','xpress_lng'])

In [34]:
import pandas
import re
condo_master_profile = pandas.read_excel('condo_bkk_info_110717.xlsx')

In [35]:
condo_master_profile['bts_distance'] = condo_master_profile['bts_distance'].astype(str)
condo_master_profile['mrt_distance'] = condo_master_profile['mrt_distance'].astype(str)
condo_master_profile['xpress_way_distance'] = condo_master_profile['xpress_way_distance'].astype(str)

In [37]:
condo_master_profile.ix[:,-20:]

,lng,floor,tower,yearbuilt,current_asking_price_per_sqm,asking_price_chg_yoy,gross_rental_yield,units,nearest_bts,bts_distance,bts_lat,bts_lng,nearest_mrt,mrt_distance,mrt_lat,mrt_lng,nearest_xpress_way,xpress_way_distance,xpress_lat,xpress_lng
0,100.440844,8,1,1996,0,0,0.00,151,Wutthakat,5.4,13.713008,100.468938,NaN,0.0,NaN,NaN,Bang Khun Thian Toll Plaza-Inbound 1,6.9,13.640494,100.412336
1,100.408137,12,1,1992,0,0,0.00,437,Wutthakat,9.4,13.713008,100.468938,NaN,0.0,NaN,NaN,Bang Khun Thian Toll Plaza-Outbound 1,4.0,13.641176,100.411155
2,100.417707,9,3,1998,0,0,0.00,367,Wutthakat,8.4,13.713008,100.468938,NaN,0.0,NaN,NaN,Bang Khun Thian Toll Plaza-Inbound 1,6.0,13.640494,100.412336
3,100.632322,5,5,1991,0,0,0.00,834,NaN,0.0,NaN,NaN,Sutthisan,8.1,13.788920,100.574175,Lat Phrao Toll Plaza,4.2,13.792900,100.611296
4,100.649395,8,2,2011,52679,-5.87,5.49,471,Phra Khanong,10.0,13.715207,100.591366,Huai Khwang,9.8,13.778456,100.574255,Lat Phrao Toll Plaza,5.8,13.792900,100.611296
5,100.628570,12,1,1994,0,0,0.00,587,Phra Khanong,7.9,13.715207,100.591366,NaN,0.0,NaN,NaN,Srinagarindra Toll Plaza,3.3,13.743164,100.611579
6,100.615507,16,1,1995,31129,0,0.00,140,Phra Khanong,6.1,13.715207,100.591366,NaN,0.0,NaN,NaN,Srinagarindra Toll Plaza,2.1,13.743164,100.611579
7,100.638799,8,1,2016,79145,–,5.90,270,NaN,0.0,NaN,NaN,Sutthisan,8.5,13.788920,100.574175,Lat Phrao Toll Plaza,4.7,13.792900,100.611296
8,100.638876,10,2,1991,73913,-17.6,8.26,560,NaN,0.0,NaN,NaN,Sutthisan,8.5,13.788920,100.574175,Lat Phrao Toll Plaza,4.7,13.792900,100.611296
9,100.646280,9,1,1997,0,0,0.00,170,NaN,0.0,NaN,NaN,Sutthisan,9.7,13.788920,100.574175,Lat Phrao Toll Plaza,5.9,13.792900,100.611296


In [45]:
#i=0
#for condo in condo_url:
    
i=1650
for condo in condo_url[1650:1700]:
    print('URL %i: %s' % (i,condo))
    try:
        condopage_parser = BeautifulSoup(get_page(condo),'html.parser')
        name = re.sub('\n','',condopage_parser.find("div", {"class":"project-header-name"}).get_text()).strip()
        province = re.sub('\n','',condopage_parser.find_all("span", {"property":"name"})[0].get_text()).strip()
        district = re.sub('\n','',condopage_parser.find_all("span", {"property":"name"})[1].get_text()).strip()
        print(name)
        address = re.sub('\n','',condopage_parser.find("span", {"itemprop":"streetAddress"}).get_text()).strip()
        lat = condopage_parser.find("meta", {"itemprop":"latitude"}).get("content")
        lng = condopage_parser.find("meta", {"itemprop":"longitude"}).get("content")
        
        ### Find number of units from Free Text in property description
        unit_str = str(condopage_parser.find("div", {"class":"property-description__content"}).find_all("p"))
        if unit_str.find('floors and includes') > 0:
            units = unit_str[unit_str.find('floors and includes')+20:unit_str.find(' units')]
        else:
            units = 0
        
        ### Check error in case of no available building info
        if condopage_parser.find("li", {"class":"project-header-year"}):
            built_year = condopage_parser.find("li", {"class":"project-header-year"}).find("span").get_text()
        else:
            built_year = 0
        if condopage_parser.find("li", {"class":"project-header-floor"}):
            num_floor = condopage_parser.find("li", {"class":"project-header-floor"}).find("span").get_text()
        else:
            num_floor = 0
        if condopage_parser.find("li", {"class":"project-header-tower"}): 
            num_tower = condopage_parser.find("li", {"class":"project-header-tower"}).find("span").get_text()
        else:
            num_tower = 0
        ### Check error in case of no available asking price data
        if condopage_parser.find("div", {"class":"indicator__amount"}):
            price = condopage_parser.find("div", {"class":"indicator__amount"}).find("span").get('data-money')
            price_chg = re.sub('\n','',condopage_parser.find_all("div", {"class":"indicator__amount"})[2].get_text()).strip()
            rental_chg = re.sub('\n','',condopage_parser.find_all("div", {"class":"indicator__amount"})[3].get_text()).strip()
        else:
            price = 0
            price_chg = 0
            rental_chg = 0
            
        ## Surrounding    
        k = 0
        bts_station = []
        bts_distance = []
        bts_lat = []
        bts_lng = []
        mrt_station = []
        mrt_distance = []
        mrt_lat = []
        mrt_lng = []
        express_way = []
        express_way_distance = []
        express_lat = []
        express_lng = []
        nearest_place = condopage_parser.find_all("div", {"class":"media neighborhood-destination"})
        while k < len(nearest_place):    
            if str(nearest_place[k]).find('bts') > 0:
                bts_station.append(nearest_place[k].find("h4", {"class":"media-heading"}).contents[0])
                distance_txt = str(nearest_place[k].find("small"))
                bts_distance.append(distance_txt[distance_txt.find('>')+1:distance_txt.find('km')-1])
                bts_lat.append(nearest_place[k].get('data-lat'))
                bts_lng.append(nearest_place[k].get('data-lng'))
            if str(nearest_place[k]).find('mrt') > 0:
                mrt_station.append(nearest_place[k].find("h4", {"class":"media-heading"}).contents[0])
                distance_txt = str(nearest_place[k].find("small"))
                mrt_distance.append(distance_txt[distance_txt.find('>')+1:distance_txt.find('km')-1])
                mrt_lat.append(nearest_place[k].get('data-lat'))
                mrt_lng.append(nearest_place[k].get('data-lng'))
            if str(nearest_place[k]).find('expressway') > 0:
                express_way.append(nearest_place[k].find("h4", {"class":"media-heading"}).contents[0])
                distance_txt = str(nearest_place[k].find("small"))
                express_way_distance.append(distance_txt[distance_txt.find('>')+1:distance_txt.find('km')-1])
                express_lat.append(nearest_place[k].get('data-lat'))
                express_lng.append(nearest_place[k].get('data-lng'))
            k+=1    
        
        if len(bts_station) == 0: bts_station = ['']
        if len(bts_distance) == 0: bts_distance = [0]
        if len(bts_lat) == 0: bts_lat = ['']
        if len(bts_lng) == 0: bts_lng = ['']
        if len(mrt_station) == 0: mrt_station = ['']
        if len(mrt_distance) == 0: mrt_distance = [0]
        if len(mrt_lat) == 0: mrt_lat = ['']
        if len(mrt_lng) == 0: mrt_lng = ['']
        if len(express_way) == 0: express_way = ['']
        if len(express_way_distance) == 0: express_way_distance = [0]
        if len(express_lat) == 0: express_lat = ['']
        if len(express_lng) == 0: express_lng = ['']    
        condo_master_profile.loc[i] = [name,district,province,address,lat,lng,
                                       num_floor,num_tower,built_year, \
                                       price,price_chg,rental_chg,units, \
                                       bts_station[0],bts_distance[0],bts_lat[0],bts_lng[0], \
                                       mrt_station[0],mrt_distance[0],mrt_lat[0],mrt_lng[0], \
                                       express_way[0],express_way_distance[0],express_lat[0],express_lng[0]]
        i+=1
    except:
        print('url error')

URL 1600: https://www.hipflat.co.th/en/bangkok/condo/supreme-ville
Supreme Ville
URL 1601: https://www.hipflat.co.th/en/bangkok/condo/t-series-place
T Series Place
URL 1602: https://www.hipflat.co.th/en/bangkok/condo/thada-court
Thada court
URL 1603: https://www.hipflat.co.th/en/bangkok/condo/the-bangkok-narathiwas-ratchanakarint
The Bangkok Narathiwas Ratchanakarint
URL 1604: https://www.hipflat.co.th/en/bangkok/condo/the-bangkok-sathon
The Bangkok Sathon
URL 1605: https://www.hipflat.co.th/en/bangkok/condo/the-diplomat-sathorn
The Diplomat Sathorn
URL 1606: https://www.hipflat.co.th/en/bangkok/condo/the-empire-place
The Empire Place
URL 1607: https://www.hipflat.co.th/en/bangkok/condo/the-hudson-sathorn-7
The Hudson Sathorn 7
URL 1608: https://www.hipflat.co.th/en/bangkok/condo/the-maple-sathonnarathiwat
The Maple Sathon-Narathiwat
URL 1609: https://www.hipflat.co.th/en/bangkok/condo/the-met
The Met
URL 1610: https://www.hipflat.co.th/en/bangkok/condo/the-natural-place-suite
The Natu

In [46]:
tmp = condo_master_profile
bts_idx = tmp[tmp.bts_distance.str.find('m')>0].index
mrt_idx = tmp[tmp.mrt_distance.str.find('m')>0].index
xp_idx = tmp[tmp.xpress_way_distance.str.find('m')>0].index
for b in bts_idx:
    tmp.loc[b,'bts_distance'] = str(float(tmp.loc[b,'bts_distance'][:tmp.loc[b,'bts_distance'].find('m')-1])/1000)

for m in mrt_idx:
    tmp.loc[m,'mrt_distance'] = str(float(tmp.loc[m,'mrt_distance'][:tmp.loc[m,'mrt_distance'].find('m')-1])/1000)

for x in xp_idx:
    tmp.loc[x,'xpress_way_distance'] = str(float(tmp.loc[x,'xpress_way_distance'][:tmp.loc[x,'xpress_way_distance'].find('m')-1])/1000)

In [47]:
# Backup Result
condo_master_profile.to_excel('condo_bkk_info_110717.xlsx')

In [52]:
condo_master_profile.ix[1600:,:]

,name,district,province,address,lat,lng,floor,tower,yearbuilt,current_asking_price_per_sqm,...,bts_lat,bts_lng,nearest_mrt,mrt_distance,mrt_lat,mrt_lng,nearest_xpress_way,xpress_way_distance,xpress_lat,xpress_lng
1600,Supreme Ville,Sathon,Bangkok,"Thanon Yen Akat, Thung Maha Mek, Sathon, Bangk...",13.714799,100.545677,8,2,1998,74355,...,,,Khlong Toei,1.6,13.722625,100.553571,Rama 4-2 Toll Plaza,1.5,13.723894,100.552355
1601,T Series Place,Sathon,Bangkok,"579 Soi Yu Di, Khwaeng Bang Khlo, Khet Bang Kh...",13.7047093,100.5151967,5,1,2008,0,...,13.719266,100.521496,,0,,,Chan Toll Plaza,0.85,13.709384,100.518874
1602,Thada court,Sathon,Bangkok,"53/6 Thanon Yen Akat, Khwaeng Chong Nonsi, Khe...",13.7120429,100.5440455,4,1,0,0,...,13.723744,100.529523,Khlong Toei,1.9,13.722625,100.553571,River Side Toll Plaza,1.5,13.707964,100.548368
1603,The Bangkok Narathiwas Ratchanakarint,Sathon,Bangkok,"Soi Chan 16, Lane 4, Khwaeng Thung Wat Don, Kh...",13.709165,100.5330246,8,5,2005,90000,...,13.723744,100.529523,,0,,,Chan Toll Plaza,2.1,13.709384,100.518874
1604,The Bangkok Sathon,Sathon,Bangkok,"251 S Sathorn Rd, Khwaeng Yan Nawa, Khet Sathon",13.7187616,100.5202356,50,1,2016,212923,...,13.719266,100.521496,,0,,,Sathon Toll Plaza,0.29,13.716434,100.520182
1605,The Diplomat Sathorn,Sathon,Bangkok,"Soi Sathon 7, Lane 4, Khwaeng Thung Maha Mek, ...",13.720942,100.532401,38,1,2013,251464,...,13.723744,100.529523,Si Lom,1.8,13.729878,100.537105,Sathon Toll Plaza,1.9,13.716434,100.520182
1606,The Empire Place,Sathon,Bangkok,"Naradhiwas Rajanagarindra Road, Khwaeng Thung ...",13.71903,100.531526,45,1,2008,132493,...,13.723744,100.529523,,0,,,Sathon Toll Plaza,1.7,13.716434,100.520182
1607,The Hudson Sathorn 7,Sathon,Bangkok,"Soi Sathon 7, Lane 3, Khwaeng Thung Maha Mek, ...",13.721326,100.533622,12,1,2014,148750,...,13.723744,100.529523,Si Lom,1.8,13.729878,100.537105,Sathon Toll Plaza,1.9,13.716434,100.520182
1608,The Maple Sathon-Narathiwat,Sathon,Bangkok,"Kusonthong, Thung Wat Don, Sathon, Bangkok 10500",13.712083,100.528646,12,1,2001,78507,...,13.719266,100.521496,,0,,,Chan Toll Plaza,1.5,13.709384,100.518874
1609,The Met,Sathon,Bangkok,"Thanon Sathon Tai, Khwaeng Thung Maha Mek, Khe...",13.722945,100.53425,66,1,2009,177054,...,13.723744,100.529523,Si Lom,1.4,13.729878,100.537105,Sathon Toll Plaza,1.9,13.716434,100.520182
